<table style="border: none" align="left">
   <tr style="border: none">
      <th style="border: none"><font face="verdana" size="5" color="black"><b>IBM Watson Machine Learning</b></th>
      <th style="border: none"><img src="https://github.com/pmservice/customer-satisfaction-prediction/blob/master/app/static/images/ml_icon_gray.png?raw=true" alt="Watson Machine Learning icon" height="40" width="40"></th>
   </tr>
   <tr style="border: none">
       <th style="border: none"><img src="https://github.com/pmservice/wml-sample-models/blob/master/spark/product-line-prediction/images/products_graphics.png?raw=true" alt="Icon"> </th>
   </tr>
</table>

In [1]:
import ibmos2spark

# @hidden_cell
credentials = {
    'auth_url': 'https://identity.open.softlayer.com',
    'project_id': 'eea2e49079a647e8b764f3a05e99d65a',
    'region': 'dallas',
    'user_id': '90ae5e017473456898bee26a82476917',
    'username': 'member_ca09aa11e41fecadfadc26bec87beb2041932b1b',
    'password': 'I^dY4wgK*CPqd?4L'
}

configuration_name = 'os_2f5406adacfa47998be8110aac9b81fc_configs'
bmos = ibmos2spark.bluemix(sc, credentials, configuration_name)

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
    
df = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(bmos.url('Weatherrelatedtrends', 'fulldata_correct_cleaned.csv'))

In [2]:
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import IntegerType

for name in df.columns:
    udf = UserDefinedFunction(lambda x: int(x), IntegerType())
    df = df.select(*[udf(column).alias(name) if column == name else column for column in df.columns])

In [3]:
df.printSchema()

root
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- mintemp: integer (nullable = true)
 |-- maxtemp: integer (nullable = true)
 |-- minprecip: integer (nullable = true)
 |-- trips: integer (nullable = true)



In [4]:
print "Number of records: " + str(df.count())

Number of records: 36


In [5]:
splitted_data = df.randomSplit([0.8, 0.18, 0.02], 24)
train_data = splitted_data[0]
test_data = splitted_data[1]
predict_data = splitted_data[2]

print "Number of training records: " + str(train_data.count())
print "Number of testing records : " + str(test_data.count())
print "Number of prediction records : " + str(predict_data.count())

Number of training records: 29
Number of testing records : 7
Number of prediction records : 0


In [6]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, IndexToString, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline, Model

In [8]:
column_names = df.columns[:-1]
vectorAssembler_features = VectorAssembler(inputCols=column_names, outputCol="features")

In [9]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(labelCol="trips", featuresCol="features", maxIter=100, regParam=0.3, elasticNetParam=0.8)

In [10]:
pipeline_lr = Pipeline(stages=[vectorAssembler_features, lr])

In [11]:
model_rf = pipeline_lr.fit(train_data)

In [12]:
predictions = model_rf.transform(test_data)
evaluatorRF = MulticlassClassificationEvaluator(labelCol="trips", predictionCol="prediction", metricName="accuracy")
accuracy = evaluatorRF.evaluate(predictions)

# print("Accuracy = %g" % accuracy)
# print("Test Error = %g" % (1.0 - accuracy))

In [13]:
from repository.mlrepositoryclient import MLRepositoryClient
from repository.mlrepositoryartifact import MLRepositoryArtifact

In [15]:
wml_credentials={
  "url": "https://ibm-watson-ml.mybluemix.net", #"https://palblyp.pmservice.ibmcloud.com/pm/v1",
  "access_key": "iQN5vdcSUVamKjcYutsI5UJd7+O01cyuRc8aBhY25/FoPlWCLCRcHsQyCfqTGli3HxGxQ3pIogjgEOjN0TGDTcL0h32gVzPkwMbmHXNpi+FQYUqQmv73SQJrb1WXWeZv",
  "username": "03a4d00f-ffef-42a1-89fe-8975c12b2604",
  "password": "5f965040-9b3f-4d48-b9ff-83060a36d318",
  "instance_id": "0cd67476-c586-4ddd-8200-ba7540a1b291"
}

In [16]:
ml_repository_client = MLRepositoryClient(wml_credentials['url'])
ml_repository_client.authorize(wml_credentials['username'], wml_credentials['password'])

In [17]:
model_artifact = MLRepositoryArtifact(model_rf, training_data=train_data, name="Visitor number prediction")

In [21]:
saved_model = ml_repository_client.models.save(model_artifact)

In [19]:
saved_model.meta.available_props()

['inputDataSchema',
 'evaluationMetrics',
 'pipelineVersionHref',
 'modelVersionHref',
 'trainingDataRef',
 'pipelineType',
 'creationTime',
 'lastUpdated',
 'label',
 'authorEmail',
 'trainingDataSchema',
 'authorName',
 'version',
 'modelType',
 'runtime',
 'evaluationMethod']

In [20]:
print "modelType: " + saved_model.meta.prop("modelType")
print "trainingDataSchema: " + str(saved_model.meta.prop("trainingDataSchema"))
print "creationTime: " + str(saved_model.meta.prop("creationTime"))
print "modelVersionHref: " + saved_model.meta.prop("modelVersionHref")
print "label: " + saved_model.meta.prop("label")

modelType: sparkml-model-2.0
trainingDataSchema: {u'fields': [{u'nullable': True, u'type': u'integer', u'name': u'month', u'metadata': {}}, {u'nullable': True, u'type': u'integer', u'name': u'year', u'metadata': {}}, {u'nullable': True, u'type': u'integer', u'name': u'mintemp', u'metadata': {}}, {u'nullable': True, u'type': u'integer', u'name': u'maxtemp', u'metadata': {}}, {u'nullable': True, u'type': u'integer', u'name': u'minprecip', u'metadata': {}}, {u'nullable': True, u'type': u'integer', u'name': u'trips', u'metadata': {}}], u'type': u'struct'}
creationTime: 2017-09-16 23:36:16.155000+00:00
modelVersionHref: https://ibm-watson-ml.mybluemix.net/v2/artifacts/models/faca4f06-0fef-4549-a660-6363a363b993/versions/a90dd515-f6b8-4336-964f-645e910ac749
label: trips


**Tip**: **modelVersionHref** is our model unique indentifier in the Watson Machine Learning repository.

### 4.2: Load model

In this subsection you will learn how to load back saved model from specified instance of Watson Machine Learning.

In [50]:
loadedModelArtifact = ml_repository_client.models.get(saved_model.uid)

You can print for example model name to make sure that model has been loaded correctly.

In [51]:
print str(loadedModelArtifact.name)

Visitor number prediction


As you can see the name is correct. You have already learned how save and load the model from Watson Machine Learning repository.

<a id="visualization"></a>
## 5. Predict locally and visualize

In this section you will learn how to score test data using loaded model and visualize the prediction results with plotly package.

### 5.1: Make local prediction using previously loaded model and test data

In this subsection you will score *predict_data* data set.

In [52]:
predictions = loadedModelArtifact.model_instance().transform(predict_data)

Preview the results by calling the *show()* method on the predictions DataFrame.

In [26]:
predictions.show(5)

+-----------------+------+---+--------------+----------+-----+-------------+---------+-----------------+------------------+--------------------+--------------------+----------+--------------------+
|     PRODUCT_LINE|GENDER|AGE|MARITAL_STATUS|PROFESSION|label|PROFESSION_IX|GENDER_IX|MARITAL_STATUS_IX|          features|       rawPrediction|         probability|prediction|      predictedLabel|
+-----------------+------+---+--------------+----------+-----+-------------+---------+-----------------+------------------+--------------------+--------------------+----------+--------------------+
|Camping Equipment|     F| 20|        Single|     Other|  0.0|          0.0|      1.0|              1.0|[1.0,20.0,1.0,0.0]|[5.54119162527029...|[0.27705958126351...|       1.0|Personal Accessories|
|Camping Equipment|     F| 20|        Single|     Other|  0.0|          0.0|      1.0|              1.0|[1.0,20.0,1.0,0.0]|[5.54119162527029...|[0.27705958126351...|       1.0|Personal Accessories|
|Camping E

By tabulating a count, you can see which product line is the most popular.

In [27]:
predictions.select("predictedLabel").groupBy("predictedLabel").count().show(truncate=False)

+------------------------+-----+
|predictedLabel          |count|
+------------------------+-----+
|Camping Equipment       |717  |
|Golf Equipment          |61   |
|Mountaineering Equipment|75   |
|Personal Accessories    |363  |
+------------------------+-----+



### 5.2: Sample visualization of data with Plotly package

In this subsection you will explore prediction results with Plotly, which is an online analytics and data visualization tool.

**Example**: First, you need to install required packages. You can do it by running the following code. Run it only one time.<BR><BR>
!pip install plotly --user <BR>
!pip install cufflinks==0.8.2 --user

Import Plotly and other required packages.

In [28]:
import sys
import pandas
import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
import plotly.graph_objs as go

init_notebook_mode(connected=True)
sys.path.append("".join([os.environ["HOME"]])) 

Convert the Apache Spark DataFrame to a Pandas DataFrame.

In [29]:
predictions_pdf = predictions.select("prediction", "predictedLabel", "GENDER", "AGE", "PROFESSION", "MARITAL_STATUS").toPandas()

Plot a pie chart that shows the predicted product-line interest.

In [30]:
cumulative_stats = predictions_pdf.groupby(['predictedLabel']).count()
product_data = [go.Pie(labels=cumulative_stats.index, values=cumulative_stats['GENDER'])]
product_layout = go.Layout(title='Predicted product line client interest distribution')

fig = go.Figure(data=product_data, layout=product_layout)
iplot(fig)

With this data set, you might want to do some analysis of the mean AGE per product line by using a bar chart.

In [31]:
age_data = [go.Bar(y=predictions_pdf.groupby(['predictedLabel']).mean()["AGE"], x=cumulative_stats.index)]

age_layout = go.Layout(
    title='Mean AGE per predicted product line',
    xaxis=dict(title = "Product Line", showline=False),
    yaxis=dict(title = "Mean AGE"))

fig = go.Figure(data=age_data, layout=age_layout)
iplot(fig)

Based on bar plot you created, you might make the following conclusion: The mean age for clients that are interested in golf equipment is predicted to be over 50 years old.

<a id="scoring"></a>
## 6. Deploy and score in a Cloud

In this section you will learn how to create online scoring and to score a new data record by using the Watson Machine Learning REST API. 
For more information about REST APIs, see the [Swagger Documentation](http://watson-ml-api.mybluemix.net/).

To work with the Watson Machine Leraning REST API you must generate an access token. To do that you can use the following sample code:

In [24]:
import urllib3, requests, json

headers = urllib3.util.make_headers(basic_auth='{username}:{password}'.format(username=wml_credentials['username'], password=wml_credentials['password']))
url = '{}/v3/identity/token'.format(wml_credentials['url'])
response = requests.get(url, headers=headers)
mltoken = json.loads(response.text).get('token')

### 6.1: Create online scoring endpoint

Now you can create an online scoring endpoint. 

#### Get published_models url from instance details

In [26]:
endpoint_instance = wml_credentials['url'] + "/v3/wml_instances/" + wml_credentials['instance_id']
header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + mltoken}

response_get_instance = requests.get(endpoint_instance, headers=header)

print response_get_instance
print response_get_instance.text

<Response [200]>
{"metadata":{"guid":"360c510b-012c-4793-ae3f-063410081c3e","url":"https://ibm-watson-ml.mybluemix.net/v3/wml_instances/360c510b-012c-4793-ae3f-063410081c3e","created_at":"2017-08-04T09:15:48.344Z","modified_at":"2017-09-06T13:28:29.140Z"},"entity":{"source":"Bluemix","published_models":{"url":"https://ibm-watson-ml.mybluemix.net/v3/wml_instances/360c510b-012c-4793-ae3f-063410081c3e/published_models"},"usage":{"expiration_date":"2017-10-01T00:00:00.000Z","computation_time":{"current":18},"model_count":{"limit":1000,"current":6},"prediction_count":{"current":15},"deployment_count":{"limit":1000,"current":11}},"plan_id":"0f2a3c2c-456b-40f3-9b19-726d2740b11c","status":"Active","organization_guid":"b0e61605-a82e-4f03-9e9f-2767973c084d","region":"us-south","account":{"id":"f52968f3dbbe7b0b53e15743d45e5e90","name":"Umit Cakmak's Account","type":"TRIAL"},"owner":{"ibm_id":"31000292EV","email":"umit.cakmak@pl.ibm.com","user_id":"43e0ee0e-6bfb-48fc-bcd8-d61e40d19253","country_co

In [27]:
endpoint_published_models = json.loads(response_get_instance.text).get('entity').get('published_models').get('url')

print endpoint_published_models

https://ibm-watson-ml.mybluemix.net/v3/wml_instances/360c510b-012c-4793-ae3f-063410081c3e/published_models


Execute the following sample code that uses the published_models endpoint to get deployments url.

#### Get the list of published models

In [29]:
header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + mltoken}

response_get = requests.get(endpoint_published_models, headers=header)

print response_get
print response_get.text

<Response [200]>
{"count":1,"resources":[{"metadata":{"guid":"99087b39-36ca-431d-9008-42a236adb432","url":"https://ibm-watson-ml.mybluemix.net/v3/wml_instances/360c510b-012c-4793-ae3f-063410081c3e/published_models/99087b39-36ca-431d-9008-42a236adb432","created_at":"2017-09-06T13:28:28.585Z","modified_at":"2017-09-06T13:28:28.717Z"},"entity":{"runtime_environment":"spark-2.0","author":{},"name":"Product Line Prediction","label_col":"PRODUCT_LINE","training_data_schema":{"fields":[{"metadata":{},"type":"string","name":"PRODUCT_LINE","nullable":true},{"metadata":{},"type":"string","name":"GENDER","nullable":true},{"metadata":{},"type":"integer","name":"AGE","nullable":true},{"metadata":{},"type":"string","name":"MARITAL_STATUS","nullable":true},{"metadata":{},"type":"string","name":"PROFESSION","nullable":true}],"type":"struct"},"latest_version":{"url":"https://ibm-watson-ml.mybluemix.net/v2/artifacts/models/99087b39-36ca-431d-9008-42a236adb432/versions/2723400a-6a3a-4603-80f5-4035134b3fd

#### Get published model deployment url

In [30]:
[endpoint_deployments] = [x.get('entity').get('deployments').get('url') for x in json.loads(response_get.text).get('resources') if x.get('metadata').get('guid') == saved_model.uid]

print endpoint_deployments

https://ibm-watson-ml.mybluemix.net/v3/wml_instances/360c510b-012c-4793-ae3f-063410081c3e/published_models/99087b39-36ca-431d-9008-42a236adb432/deployments


#### Create online deployment for published model

In [31]:
payload_online = {"name": "Product Line Prediction", "description": "My Cool Deployment", "type": "online"}
response_online = requests.post(endpoint_deployments, json=payload_online, headers=header)

print response_online
print response_online.text

<Response [201]>
{"metadata":{"guid":"c8ad0991-a4b2-45ec-8555-019edfcaff89","url":"https://ibm-watson-ml.mybluemix.net/v3/wml_instances/360c510b-012c-4793-ae3f-063410081c3e/published_models/99087b39-36ca-431d-9008-42a236adb432/deployments/c8ad0991-a4b2-45ec-8555-019edfcaff89","created_at":"2017-09-06T13:34:14.366Z","modified_at":"2017-09-06T13:34:16.060Z"},"entity":{"runtime_environment":"spark-2.0","name":"Product Line Prediction","scoring_url":"https://ibm-watson-ml.mybluemix.net/v3/wml_instances/360c510b-012c-4793-ae3f-063410081c3e/published_models/99087b39-36ca-431d-9008-42a236adb432/deployments/c8ad0991-a4b2-45ec-8555-019edfcaff89/online","description":"My Cool Deployment","published_model":{"author":{},"name":"Product Line Prediction","url":"https://ibm-watson-ml.mybluemix.net/v3/wml_instances/360c510b-012c-4793-ae3f-063410081c3e/published_models/99087b39-36ca-431d-9008-42a236adb432","guid":"99087b39-36ca-431d-9008-42a236adb432","created_at":"2017-09-06T13:34:14.332Z"},"model_typ

In [32]:
scoring_url = json.loads(response_online.text).get('entity').get('scoring_url')
print scoring_url

https://ibm-watson-ml.mybluemix.net/v3/wml_instances/360c510b-012c-4793-ae3f-063410081c3e/published_models/99087b39-36ca-431d-9008-42a236adb432/deployments/c8ad0991-a4b2-45ec-8555-019edfcaff89/online


Now, you can send (POST) new scoring records (new data) for which you would like to get predictions. To do that, execute the following sample code: 

In [33]:
payload_scoring = {"fields": ["GENDER","AGE","MARITAL_STATUS","PROFESSION"],"values": [["M",23,"Single","Student"],["M",55,"Single","Executive"]]}
response_scoring = requests.post(scoring_url, json=payload_scoring, headers=header)

print response_scoring.text

{
  "fields": ["GENDER", "AGE", "MARITAL_STATUS", "PROFESSION", "PRODUCT_LINE", "label", "PROFESSION_IX", "GENDER_IX", "MARITAL_STATUS_IX", "features", "rawPrediction", "probability", "prediction", "predictedLabel"],
  "values": [["M", 23, "Single", "Student", "Camping Equipment", 0.0, 6.0, 0.0, 1.0, [0.0, 23.0, 1.0, 6.0], [5.600716147152702, 6.482458372136143, 6.048004170730676, 0.20929155492307386, 1.6595297550574055], [0.2800358073576351, 0.32412291860680714, 0.3024002085365338, 0.010464577746153694, 0.08297648775287028], 1.0, "Personal Accessories"], ["M", 55, "Single", "Executive", "Camping Equipment", 0.0, 3.0, 0.0, 1.0, [0.0, 55.0, 1.0, 3.0], [6.227653744886563, 4.325198862164969, 8.031953760146816, 1.2319759269281225, 0.1832177058735289], [0.3113826872443282, 0.2162599431082485, 0.40159768800734086, 0.06159879634640614, 0.009160885293676447], 2.0, "Mountaineering Equipment"]]
}


As we can see we predict that a 23-year-old male student is interested in personal accessories (predictedLabel: Personal Accessories, prediction: 1.0). We can also see that single, 55 years old man is interested in mountaineering equipment.

<a id="summary"></a>
## 7. Summary and next steps     

 You successfully completed this notebook! You learned how to use Apache Spark machine learning as well as Watson Machine Learning for model creation and deployment. Check out our _[Online Documentation](https://console.ng.bluemix.net/docs/services/PredictiveModeling/pm_service_api_spark.html)_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors

**Lukasz Cmielowski**, PhD, is a Automation Architect and Data Scientist in IBM with a track record of developing enterprise-level applications that substantially increases clients' ability to turn data into actionable knowledge.

Copyright © 2017 IBM. This notebook and its source code are released under the terms of the MIT License.